Reformat published datasets for CysteineomeDB. Categories: Dataset Found, Reactive, Ligandable, Identified but not Ligandale, and Conditional. Note: Annotations from authors were used to determine "ligandabiliy." Note: Peptide sequences from Yang DIA were manually mapped to the uniprot fasta (2201) to find cysteine identifiers.

main directory 'data' has nine folders according to each publication

# Setup Environment

In [ ]:
import os, sys
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import csv
import matplotlib
import numpy as np
import math
from matplotlib.pyplot import figure
import Bio
from Bio import SeqIO
from functools import reduce

In [ ]:
cd = os.getcwd()
cd

In [ ]:
path_data = os.path.join(os.getcwd(), 'results')
if not os.path.exists(path_data):
    os.makedirs(path_data)

In [ ]:
# merge identifiers from all csvs

def get_new_df(dfs, dataset, col1, col2, cys):
    new_df = pd.concat(dfs)
    
    new_df = new_df[new_df[col1].str.contains("contaminant") == False]
    
    if dataset == 'kuljanin_gygi':
        new_df['proteinid'] = new_df[col1].map(lambda x: str(x).replace(' ','').split('|')[1])
        new_df['resid'] = new_df[col2].map(lambda x: 'C' + str(x).replace(' ',''))
    elif dataset == 'weerapana_cravatt':
        new_df['proteinid'] = new_df[col1].map(lambda x: str(x).replace(' ',''))
        new_df['resid'] = new_df[col2].map(lambda x: 'C' + str(x).replace(' ',''))
    elif dataset == 'backus_cravatt':
        new_df['proteinid'] = new_df['Identifier'].map(lambda x: str(x).replace(' ','').split('_')[0])
        new_df['resid'] = new_df['Identifier'].map(lambda x: str(x).replace(' ','').split('_')[-1]) 
    elif dataset == 'yan_backus':
        new_df['proteinid'] = new_df[col1].map(lambda x: str(x).replace(' ',''))
        new_df['resid'] = new_df['identifier'].map(lambda x: 'C' + str(x).replace(' ','').split('_')[-1]) 
    elif dataset == 'yang_wang':
        new_df['proteinid'] = new_df[col1]
        new_df['resid'] = new_df[col2].map(lambda x: 'C' + str(x).replace(' ',''))         
    else:
        new_df = new_df.rename(columns = {col1: 'proteinid', col2: 'resid'})
        
    new_df['cysteineid'] = new_df['proteinid'] + '_' + new_df['resid'].astype(str)
    new_df['dataset'] = dataset
    new_df['identified'] = 1
    new_df['identified_datasets'] = dataset
    
    if cys == True:
        new_df['level'] = 'cysteine'
        new_df = new_df[['level', 'cysteineid', 'proteinid', 'dataset', 'identified', 'identified_datasets']]
    else:
        new_df['level'] = 'protein'
        new_df = new_df[['level', 'proteinid', 'dataset', 'identified', 'identified_datasets']]
    new_df = new_df.drop_duplicates()
    
    return new_df

In [ ]:
# create protein identifiers
# UniProtKB

def get_pro_uniprot_identifier(master, df, dataset, category, category_datasets, col1, col2):
    if dataset == 'weerapana_cravatt':
        df['proteinid'] = df[col1].map(lambda x: str(x).replace(' ',''))
    elif dataset == 'kuljanin_gygi':
        df = df[[col1, col2]]
        df = df.drop_duplicates()
        df['proteinid'] = df[col1].map(lambda x: str(x).replace(' ','').split('|')[1])
    elif dataset == 'backus_cravatt':
        df['proteinid'] = df['Identifier'].map(lambda x: str(x).replace(' ','').split('_')[0])
        df = df[['proteinid']]
        df = df.drop_duplicates()
    elif dataset == 'yang_wang':
        df['proteinid'] = df[col1].map(lambda x: str(x).replace(' ',''))
    else:
        df = df[[col1]]
        df = df.drop_duplicates()
        df = df.rename(columns = {col1: 'proteinid'})

    df_ids = list(df['proteinid'].unique())
    
    master[category] = np.where(master['proteinid'].isin(df_ids), 1, 0)
    category_df = master[master[category] == 1]
    category_df[category  + '_datasets'] = [category_datasets] * category_df.shape[0]
    non_category_df = master[master[category] == 0]
    
    new_df = pd.concat([category_df, non_category_df])

    return new_df

In [ ]:
# create cysteine identifiers
# UniProtKB_C#

def get_cys_uniprot_identifier(master, df, dataset, category, category_datasets, col1, col2):
    if dataset == 'weerapana_cravatt':
        df['proteinid'] = df[col1].map(lambda x: str(x).replace(' ',''))
        df['resid'] = df[col2].map(lambda x: 'C' + str(x).replace(' ',''))
    elif dataset == 'kuljanin_gygi':
        df = df[[col1, col2]]
        df = df.drop_duplicates()
        df['proteinid'] = df[col1].map(lambda x: str(x).replace(' ','').split('|')[1])
        df['resid'] = df[col2].map(lambda x: 'C' + str(x).replace(' ',''))
    elif dataset == 'backus_cravatt':
        df['proteinid'] = df['Identifier'].map(lambda x: str(x).replace(' ','').split('_')[0])
        df['resid'] = df['Identifier'].map(lambda x: str(x).replace(' ','').split('_')[-1])  
        df = df[['proteinid', 'resid']]
        df = df.drop_duplicates()
    elif dataset == 'yan_backus':
        df['proteinid'] = df[col1].map(lambda x: str(x).replace(' ',''))
        df['resid'] = df[col2].map(lambda x: 'C' + str(x).replace(' ','').split('_')[-1]) 
    elif dataset == 'yang_wang':
        df['proteinid'] = df[col1].map(lambda x: str(x).replace(' ',''))
        df['resid'] = df[col2].map(lambda x: 'C' + str(x).replace(' ','')) 
    else:
        df = df[[col1, col2]]
        df = df.drop_duplicates()
        df = df.rename(columns = {col1: 'proteinid', col2: 'resid'})
        
    df['cysteineid'] = df['proteinid'] + '_' + df['resid'].astype(str)
    df_ids = list(df['cysteineid'].unique())
    
    master[category] = np.where(master['cysteineid'].isin(df_ids), 1, 0)
    category_df = master[master[category] == 1]
    category_df[category  + '_datasets'] = [category_datasets] * category_df.shape[0]
    non_category_df = master[master[category] == 0]
    
    new_df = pd.concat([category_df, non_category_df])

    return new_df

In [ ]:
def update_category(df, category, labels, datasets):
    df['category_sum'] = df[labels].sum(axis=1)
    df[category] = np.where(df['category_sum'] >= 1 , 1, 0)
    
    category_datasets = []
    for index, row in df.iterrows():
        current_datasets = []
        for i in range(len(datasets)):
            if str(row[datasets[i]]) != 'nan':
                current_datasets.append(row[datasets[i]])
        
        category_datasets.append(list_to_string(list(set(current_datasets)), ';'))
        
            
    df[category + '_datasets'] = category_datasets
    df = df.drop(columns = labels + datasets + ['category_sum'])
    
    return df

In [ ]:
def list_to_string(lst, symbol):
    return (symbol.join([str(elem) for elem in lst]))

# Extract Experimental Data

# Read Quantifying Functional Cysteines Data

In [ ]:
os.chdir(cd)
os.chdir('Functional')

In [ ]:
df_function = pd.read_csv('nature_2010.csv')
df_function = df_function[df_function['Identifier'].str.contains('Reverse') == False]
df_function = df_function[df_function['Identifier'].str.contains('contaminant') == False]

In [ ]:
weerapana_df = get_new_df([df_function], 'weerapana_cravatt', 'Protein', 'PTM', True)

In [ ]:
weerapana_cys_ids = list(weerapana_df['cysteineid'].unique())
weerapana_pro_ids = list(weerapana_df['proteinid'].unique())

In [ ]:
cys_weerapana_df = weerapana_df.copy()

In [ ]:
pro_weerapana_df = weerapana_df.copy()
pro_weerapana_df = pro_weerapana_df.drop(columns = ["cysteineid", "level"])
pro_weerapana_df = pro_weerapana_df.drop_duplicates()
pro_weerapana_df['level'] = 'protein'

In [ ]:
cys_weerapana_df.shape, pro_weerapana_df.shape

In [ ]:
weerapana_df = pd.concat([cys_weerapana_df, pro_weerapana_df])

In [ ]:
os.chdir(cd)
os.chdir('results')
weerapana_df.to_csv('weerapana_cravatt_dataset.csv', index = False)

# Read Proteome-wide Discovery Data

In [ ]:
os.chdir(cd)
os.chdir('Discovery')

In [ ]:
df_discovery_ligandable = pd.read_excel('41586_2016_BFnature18002_MOESM54_ESM.xlsx', sheet_name='Probe Targets In vitro')
df_discovery_not_ligandable = pd.read_excel('41586_2016_BFnature18002_MOESM54_ESM.xlsx', sheet_name='In vitro (not probe targets)')

In [ ]:
backus_df = get_new_df([df_discovery_ligandable, df_discovery_not_ligandable], 'backus_cravatt', 'Identifier', '', True)

In [ ]:
cys_backus_df = backus_df.copy()

In [ ]:
pro_backus_df = get_new_df([df_discovery_ligandable, df_discovery_not_ligandable], 'backus_cravatt', 'Identifier', '', False)

In [ ]:
cys_backus_df.shape, pro_backus_df.shape

In [ ]:
backus_df = pd.concat([cys_backus_df, pro_backus_df])

In [ ]:
os.chdir(cd)
os.chdir('results')
backus_df.to_csv('backus_cravatt_dataset.csv', index = False)

# Read Suzuki Data

In [ ]:
os.chdir(cd)
os.chdir('Suzuki')
os.chdir('5275142')

In [ ]:
df_suzuki = pd.read_excel('ac0c04726_si_002.xlsx', sheet_name='Suzuki_CuAAC_w_sp3')

In [ ]:
df_suzuki['protein'] = df_suzuki['identifier'].map(lambda x: str(x).split('_')[0])

In [ ]:
df_suzuki['cysteine_ptm'] = df_suzuki['cysteine_ptm'].map(lambda x: 'C' + str(x))

In [ ]:
df_suzuki_click = df_suzuki[df_suzuki['identified_in_click_dataset (0: no | 1: yes)'] == 1]
df_suzuki_suzuki = df_suzuki[df_suzuki['identified_in_suzuku_dataset (0: no | 1: yes)'] == 1]

In [ ]:
df_suzuki_ligandability = pd.read_excel('ac0c04726_si_003.xlsx', sheet_name='Liganded Cysteines')

In [ ]:
df_suzuki_ligandability['cysteine_ptm'] = df_suzuki_ligandability['Cysteine Labeled'].str.split(';')
df_suzuki_ligandability['protein'] = df_suzuki_ligandability['Proteins'].str.split(',')

In [ ]:
df_suzuki_ligandability_subset = df_suzuki_ligandability[['Sequence', 'cysteine_ptm', 'protein']]
df_suzuki_ligandability_subset = df_suzuki_ligandability_subset.set_index('Sequence').apply(pd.Series.explode).reset_index()
df_suzuki_ligandability_subset = df_suzuki_ligandability_subset.drop(columns = 'Sequence')

In [ ]:
df_suzuki_ligandability_explode = df_suzuki_ligandability[['Sequence', 'cysteine_ptm', 'protein']]
df_suzuki_ligandability_explode = df_suzuki_ligandability_explode.set_index('Sequence').apply(pd.Series.explode).reset_index()
df_suzuki_ligandability_explode['cysteineid'] = df_suzuki_ligandability_explode['protein'] + '_' + df_suzuki_ligandability_explode['cysteine_ptm']

In [ ]:
merge_df_suzuki_ligandability = df_suzuki_ligandability.drop(columns = ['cysteine_ptm', 'protein'])
df_suzuki_ligandability_merged = pd.merge(df_suzuki_ligandability_explode, merge_df_suzuki_ligandability , on = 'Sequence', how = 'left')

In [ ]:
cao_df = get_new_df([df_suzuki, df_suzuki_ligandability_merged], 'cao_backus', 'protein', 'cysteine_ptm', True)

In [ ]:
cys_cao_df = cao_df.copy()

In [ ]:
pro_cao_df = get_new_df([df_suzuki, df_suzuki_ligandability_subset], 'cao_backus', 'protein', 'cysteine_ptm', False)

In [ ]:
cys_cao_df.shape, pro_cao_df.shape

In [ ]:
cao_df = pd.concat([cys_cao_df, pro_cao_df])

In [ ]:
os.chdir(cd)
os.chdir('results')
cao_df.to_csv('cao_backus_dataset.csv', index = False)

# Read Mapping Data

In [ ]:
os.chdir(cd)
os.chdir('Mapping')

In [ ]:
df_mapping = pd.read_excel('msb20209840-sup-0020-datasetev18.xlsx', sheet_name='Sheet2')

In [ ]:
palafox_df = get_new_df([df_mapping], 'palafox_backus', 'Protein ID', 'Cysteine', True)

In [ ]:
cys_palafox_df = palafox_df.copy()

In [ ]:
pro_palafox_df = palafox_df.copy()
pro_palafox_df = pro_palafox_df.drop(columns = ["cysteineid", "level"])
pro_palafox_df = pro_palafox_df.drop_duplicates()
pro_palafox_df['level'] = 'protein'

In [ ]:
cys_palafox_df.shape, pro_palafox_df.shape

In [ ]:
palafox_df = pd.concat([cys_palafox_df, pro_palafox_df])

In [ ]:
os.chdir(cd)
os.chdir('results')
palafox_df.to_csv('palafox_backus_dataset.csv', index = False)

# Read SP3 Data

In [ ]:
os.chdir(cd)
os.chdir('SP3')

In [ ]:
df_sp3 = pd.read_excel('cbic202000870-sup-0001-table_s4.xlsx', sheet_name='Aggregate_Identifiers')

In [ ]:
def split_sp3(df):
    
    new_df = df[['identifier', 'protein']]
    new_df = new_df.drop_duplicates()
    
    df_cols = df.columns.to_list()
    grouped_cols = sorted(df_cols[4:])

    for i in range(0, len(grouped_cols), 2):
        name = grouped_cols[i].split('-')[0]
        new_df[name] = df[grouped_cols[i]].astype(int) + df[grouped_cols[i+1]].astype(int)
    
    return new_df
 

In [ ]:
df_sp3_split = split_sp3(df_sp3)

## Read ligandability data

In [ ]:
df_sp3_lig = pd.read_excel("cbic202000870-sup-0001-table_s6_modified.xlsx")

In [ ]:
df_sp3_lig = df_sp3_lig[df_sp3_lig['Protein'] != 'conta.minant']

In [ ]:
df_sp3_lig['protein'] = df_sp3_lig['Protein'].str.split(',')

In [ ]:
df_sp3_lig['resid'] = df_sp3_lig['PTM'].str.split(',')

In [ ]:
df_sp3_lig = df_sp3_lig.explode('protein')

In [ ]:
df_sp3_lig = df_sp3_lig.explode('resid')

In [ ]:
df_sp3_lig['resid'] = df_sp3_lig['resid'].map(lambda x: str(x)[1:])

In [ ]:
df_sp3_lig = df_sp3_lig.drop(columns = ['PTM', 'Protein'])
df_sp3_lig = df_sp3_lig.drop_duplicates()

In [ ]:
df_sp3_lig['identifier'] = df_sp3_lig['protein'] + '_' + df_sp3_lig['resid']

In [ ]:
yan_df = get_new_df([df_sp3, df_sp3_lig], 'yan_backus', 'protein', 'identifier', True)

In [ ]:
yan_df

In [ ]:
cys_yan_df = yan_df.copy()

In [ ]:
pro_yan_df = get_new_df([df_sp3, df_sp3_lig], 'yan_backus', 'protein', 'identifier', False)

In [ ]:
cys_yan_df.shape, pro_yan_df.shape

In [ ]:
yan_df = pd.concat([cys_yan_df, pro_yan_df])

In [ ]:
os.chdir(cd)
os.chdir('results')
yan_df.to_csv('yan_backus_dataset.csv', index = False)

# Read T-cell Data

In [ ]:
os.chdir(cd)
os.chdir('Tcell')

In [ ]:
df_tcell = pd.read_excel('NIHMS1616434-supplement-mmc4.xlsx', sheet_name='Table S6_Master Table', header = [5])

In [ ]:
df_tcell['Residue'] = df_tcell['Residues'].str.split(',')
df_tcell_split = df_tcell.explode('Residue')

In [ ]:
vinogradova_df = get_new_df([df_tcell_split], 'vinogradova_cravatt', 'Uniprot', 'Residue', True)

In [ ]:
cys_vinogradova_df = vinogradova_df.copy()

In [ ]:
pro_vinogradova_df = get_new_df([df_tcell], 'vinogradova_cravatt', 'Uniprot', 'Residue', False)

In [ ]:
cys_vinogradova_df.shape, pro_vinogradova_df.shape

In [ ]:
vinogradova_df = pd.concat([cys_vinogradova_df, pro_vinogradova_df])

In [ ]:
os.chdir(cd)
os.chdir('results')
vinogradova_df.to_csv('vinogradova_cravatt_dataset.csv', index = False)

# Read SLCABPP Data

In [ ]:
os.chdir(cd)
os.chdir('SLCABPP')
os.chdir('NIHMS1660575-supplement-9_supplementary_tables')

In [ ]:
df_hct = pd.read_excel('41587_2020_778_S6_ESM.xlsx', sheet_name='SLC-ABPP HCT116 Screen')
df_hek = pd.read_excel('41587_2020_778_S7_ESM.xlsx', sheet_name='SLC-ABPP HEK293T SCreen')
df_patu = pd.read_excel('41587_2020_778_S8_ESM.xlsx', sheet_name='SLC-ABPP PaTu-8988T Screen')

In [ ]:
df_scout = pd.read_excel('41587_2020_778_S4_ESM.xlsx', sheet_name='Scout Profiling')

In [ ]:
kuljanin_df = get_new_df([df_hct, df_hek, df_patu, df_scout], 'kuljanin_gygi', 'Uniprot ID', 'Site Position', True)

In [ ]:
cys_kuljanin_df = kuljanin_df.copy()

In [ ]:
pro_kuljanin_df = get_new_df([df_hct, df_hek, df_patu, df_scout], 'kuljanin_gygi', 'Uniprot ID', 'Site Position', False)

In [ ]:
pro_kuljanin_df = update_category(pro_kuljanin_df, 'ligandable', 
                         ['ligandable_1', 'ligandable_2', 'ligandable_3', 'ligandable_4'], 
                         ['ligandable_1_datasets', 'ligandable_2_datasets', 'ligandable_3_datasets', 'ligandable_4_datasets'])

In [ ]:
cys_kuljanin_df.shape, pro_kuljanin_df.shape

In [ ]:
kuljanin_df = pd.concat([cys_kuljanin_df, pro_kuljanin_df])

In [ ]:
os.chdir(cd)
os.chdir('results')
kuljanin_df.to_csv('kuljanin_gygi_dataset.csv', index = False)

# Read DIA Data

In [ ]:
def get_uniprot_dict(directory, file):
    os.chdir(directory)
    
    uniprot_dict = {}
    for record in SeqIO.parse(file, "fasta"):
        protein = str(record.id).split('|')[1]
        uniprot_dict[protein] = record.seq
    
    os.chdir(cd)
    return uniprot_dict

In [ ]:
uniprot_dict = get_uniprot_dict('/Users/lisamarieboatner/Dropbox/Backus/master/uniprot/data/220505', '2201_uniprot.fasta')
# uniprot_dict = get_uniprot_dict('C:\\Users\\Onee-sama\\Dropbox\\Backus\\master\\uniprot\\data\\220724', '2207_uniprot.fasta')

In [ ]:
def get_residue_aa(identifier, peptide, uniprot_dict, uniprot_aa):
    if (peptide in str(uniprot_dict[identifier])):
        correct_aa = str(uniprot_dict[identifier]).index(peptide) + (int(uniprot_aa))
        return int(correct_aa)
    else:
        return '--'

In [ ]:
def get_cysetine_ids(df, uniprot_dict):
    cysteines = []
    missing_protein_ids = []
    missing_peptide_ids = []
    
    for index, row in df.iterrows():
        protein = row['Protein'].strip()
        modified_peptide = row['Peptides']
        modified_aa = modified_peptide.index('*')
        unmodified_peptide = modified_peptide.replace('*', '')

        if (protein not in uniprot_dict.keys()):
            correct_aa = '--'
            missing_protein_ids.append(protein)
            missing_peptide_ids.append(modified_peptide)
        else:
            correct_aa = get_residue_aa(protein, unmodified_peptide, uniprot_dict, modified_aa)
    
        cysteines.append(correct_aa)
        
    return cysteines, missing_protein_ids, missing_peptide_ids

## Read reactivity data 

In [ ]:
os.chdir(cd)
os.chdir('DIA')

In [ ]:
df_dia_reactivity = pd.read_excel('https://www.dropbox.com/s/5vwm2p4qsbw4599/ja1c11053_si_002.xlsx?dl=1', sheet_name='DIA-ABPP')

In [ ]:
df_dia_reactivity['Protein'] = df_dia_reactivity['Proteins'].str.split(',')
df_dia_reactivity_split = df_dia_reactivity.explode('Protein')

In [ ]:
dia_reactivity_cysteine_ids, dia_reactivity_missing_protein_ids, missing_peptide_ids = get_cysetine_ids(df_dia_reactivity_split, uniprot_dict)

In [ ]:
len(dia_reactivity_missing_protein_ids)

There are 37 proteins identified in DIA reactivity experiment that were not found in 2207 fasta.

In [ ]:
len(dia_reactivity_missing_protein_ids), len(missing_peptide_ids)

In [ ]:
missing_id_df = pd.DataFrame()

In [ ]:
missing_id_df['proteinid'] = dia_reactivity_missing_protein_ids

In [ ]:
missing_id_df['peptide'] = missing_peptide_ids

In [ ]:
missing_id_df.to_csv('dia_missing_ids.csv', index = False)

In [ ]:
df_dia_reactivity_split['Cysteine'] = dia_reactivity_cysteine_ids
df_dia_reactivity_split = df_dia_reactivity_split[df_dia_reactivity_split['Cysteine'] != '--']

## Read ligandability data

In [ ]:
os.chdir(cd)
os.chdir('DIA')

In [ ]:
df_dia_ligandability = pd.read_excel('ja1c11053_si_003.xlsx', sheet_name='all cysteines')

In [ ]:
df_dia_ligandability['Protein'] = df_dia_ligandability['Proteins'].str.split(',')
df_dia_ligandability_split = df_dia_ligandability.explode('Protein')

In [ ]:
dia_ligandability_cysteine_ids, dia_ligandability_missing_protein_ids = get_cysetine_ids(df_dia_ligandability_split, uniprot_dict)

In [ ]:
len(dia_ligandability_missing_protein_ids)

There are 113 proteins identified in DIA ligandability experiment that were not found in 2201 fasta.

In [ ]:
df_dia_ligandability_split['Cysteine'] = dia_ligandability_cysteine_ids
df_dia_ligandability_split = df_dia_ligandability_split[df_dia_ligandability_split['Cysteine'] != '--']

In [ ]:
yang_df = get_new_df([df_dia_reactivity_split, df_dia_ligandability_split], 'yang_wang', 'Protein', 'Cysteine', True)

In [ ]:
cys_yang_df = yang_df.copy()

In [ ]:
pro_yang_df = get_new_df([df_dia_reactivity_split, df_dia_ligandability_split], 'yang_wang', 'Protein', 'Cysteine', False)

In [ ]:
cys_yang_df.shape, pro_yang_df.shape

In [ ]:
yang_df = pd.concat([cys_yang_df, pro_yang_df])

In [ ]:
os.chdir(cd)
os.chdir('results')
yang_df.to_csv('yang_wang_dataset.csv', index = False)

# Read Biotin Cleavable Tags Data

In [ ]:
os.chdir(cd)
os.chdir('Biotin')

In [ ]:
df_tag1_sheet_a = pd.read_excel('pr2c00174_si_002.xlsx', sheet_name='Workflow A')
df_tag1_sheet_b = pd.read_excel('pr2c00174_si_002.xlsx', sheet_name='Workflow B')
df_tag1_sheet_c = pd.read_excel('pr2c00174_si_002.xlsx', sheet_name='Workflow C')

In [ ]:
df_tag2_sheet_a = pd.read_excel('pr2c00174_si_003.xlsx', sheet_name='Workflow A')
df_tag2_sheet_b = pd.read_excel('pr2c00174_si_003.xlsx', sheet_name='Workflow B')
df_tag2_sheet_c = pd.read_excel('pr2c00174_si_003.xlsx', sheet_name='Workflow C')

In [ ]:
df_tag3_sheet_a = pd.read_excel('pr2c00174_si_004.xlsx', sheet_name='Workflow A')
df_tag3_sheet_b = pd.read_excel('pr2c00174_si_004.xlsx', sheet_name='Workflow B')
df_tag3_sheet_c = pd.read_excel('pr2c00174_si_004.xlsx', sheet_name='Workflow C')

In [ ]:
df_tag4_sheet_a = pd.read_excel('pr2c00174_si_005.xlsx', sheet_name='Workflow A')
df_tag4_sheet_b = pd.read_excel('pr2c00174_si_005.xlsx', sheet_name='Workflow B')
df_tag4_sheet_c = pd.read_excel('pr2c00174_si_005.xlsx', sheet_name='Workflow C')

In [ ]:
df_tag5_sheet_a = pd.read_excel('pr2c00174_si_006.xlsx', sheet_name='Workflow A')
df_tag5_sheet_b = pd.read_excel('pr2c00174_si_006.xlsx', sheet_name='Workflow B')
df_tag5_sheet_c = pd.read_excel('pr2c00174_si_006.xlsx', sheet_name='Workflow C')

In [ ]:
concat_li_df = pd.concat([df_tag1_sheet_a, df_tag1_sheet_b, df_tag1_sheet_c,
                     df_tag2_sheet_a, df_tag2_sheet_b, df_tag2_sheet_c,
                     df_tag3_sheet_a, df_tag3_sheet_b, df_tag3_sheet_c,
                     df_tag4_sheet_a, df_tag4_sheet_b, df_tag4_sheet_c,
                     df_tag5_sheet_a, df_tag5_sheet_b, df_tag5_sheet_c,
                    ])

In [ ]:
li_df = get_new_df([df_tag1_sheet_a, df_tag1_sheet_b, df_tag1_sheet_c,
                     df_tag2_sheet_a, df_tag2_sheet_b, df_tag2_sheet_c,
                     df_tag3_sheet_a, df_tag3_sheet_b, df_tag3_sheet_c,
                     df_tag4_sheet_a, df_tag4_sheet_b, df_tag4_sheet_c,
                     df_tag5_sheet_a, df_tag5_sheet_b, df_tag5_sheet_c,
                    ], 'li_yang', 'Uniprot accession #', 'Site', True)

In [ ]:
cys_li_df = li_df.copy()

In [ ]:
pro_li_df = get_new_df([df_tag1_sheet_a, df_tag1_sheet_b, df_tag1_sheet_c,
                     df_tag2_sheet_a, df_tag2_sheet_b, df_tag2_sheet_c,
                     df_tag3_sheet_a, df_tag3_sheet_b, df_tag3_sheet_c,
                     df_tag4_sheet_a, df_tag4_sheet_b, df_tag4_sheet_c,
                     df_tag5_sheet_a, df_tag5_sheet_b, df_tag5_sheet_c,
                    ], 'li_yang', 'Uniprot accession #', 'Site', False)

In [ ]:
pro_li_df = get_new_df([df_tag1_sheet_a, df_tag1_sheet_b, df_tag1_sheet_c
                    ], 'li_yang', 'Uniprot accession #', 'Site', False)

In [ ]:
cys_li_df.shape, pro_li_df.shape

In [ ]:
li_df = pd.concat([cys_li_df, pro_li_df])

In [ ]:
os.chdir(cd)
os.chdir('results')
li_df.to_csv('li_yang_dataset.csv', index = False)

# Transform

In [ ]:
def get_exp_df(dfs):
    new_df = pd.concat(dfs)
    
    new_df = new_df.rename(columns = {'identifier': 'cysteineid'})

    id_df = new_df[['cysteineid', 'proteinid', 'resid']]
    id_df = id_df.drop_duplicates()
    
    return new_df, id_df

In [ ]:
def get_col_names(df, name, find_in, multi):
    df_cols = df.columns.to_list()
    
    if find_in == True:
        results = [item for item in df_cols if name in item]
    else:
        if multi == True:
            results = [item for item in df_cols if item[0].isupper()]
        else:
            results = [item for item in df_cols if name not in item]
    
    return results

In [ ]:
def get_single_labels(df, col1, dataset_cols, reactive_cols, ligandable_cols, not_ligandable_cols, experiment_cols, new_auth):
    
    ids = []
    
    ds_labels, exp_labels, rxt_labels, lig_labels, nolig_labels = [], [], [], [], []
    ds_vals, exp_vals, rxt_vals, lig_vals, nolig_vals = [], [], [], [], []
   
    for name, group in df:
        count = 0
        for index, row in group.iterrows():
            ds_labels += search(dataset_cols, row)
            ds_vals += search_results(dataset_cols, row)
            rxt_labels += search(reactive_cols, row)
            rxt_vals += search_results(reactive_cols, row)
            lig_labels += search(ligandable_cols, row)
            lig_vals += search_results(ligandable_cols, row)
            nolig_labels += search(not_ligandable_cols, row)
            nolig_vals += search_results(not_ligandable_cols, row)
            exp_labels += search(experiment_cols, row)
            exp_vals += search_results(experiment_cols, row)
            
        ids.append(name)
    
    auth = [new_auth] * len(ids)
    zipped = list(zip(ids, auth, 
                      ds_labels, ds_vals, 
                      rxt_labels, rxt_vals, 
                      lig_labels, lig_vals, 
                      nolig_labels, nolig_vals,
                      exp_labels, exp_vals))
    
    new_df = pd.DataFrame(zipped, columns = [col1, 'dataset', 
                                            'identified', 'identified_datasets', 
                                            'reactive', 'reactive_datasets',
                                            'ligandable', 'ligandable_datasets',
                                            'not_ligandable', 'not_ligandable_datasets',
                                            'conditional', 'conditional_datasets'])
    
    return new_df

In [ ]:
def search(search_cols, row):
    found = np.nan
    for i in range(len(search_cols)):
        current = row[search_cols[i]]
        if current == 'yes':
            found = 'yes'
    return [found]

In [ ]:
def search_results(search_cols, row):
    results = []
    for i in range(len(search_cols)):
        current = row[search_cols[i]]
        if (current == 'yes') & (search_cols[i] not in results):
            results.append(search_cols[i])

    return [list_to_string(results, ';')]

In [ ]:
def list_to_string(lst, symbol):
    return (symbol.join([str(elem) for elem in lst]))

In [ ]:
def get_merged_df (dfs, id_df, cys):
    merged_df = id_df.copy()
    for i in range(len(dfs)):
        if cys == True:
            merged_df = pd.merge(merged_df, current_df, on = 'cysteineid', how = 'left')
            
        else:
            merged_df = pd.merge(merged_df, current_df, on = 'proteinid', how = 'left')
        
    return merged_df

In [ ]:
def get_single_dataset(df, old_auth, new_auth):

    old_colnames = df.columns.to_list()
    new_colnames = {}
    
    for i in range(len(old_colnames)):
        current_colname = old_colnames[i]
        
        if old_auth in current_colname:
            new_colname = current_colname.replace(old_auth + '_', "")
            
            if "identified" in current_colname:
                new_colname = new_colname.replace("identified", "not_ligandable")
            if "dataset" in current_colname:
                new_colname = new_colname.replace("dataset", "identified")
            new_colname = new_colname.title()
            
            new_colnames[current_colname] = new_colname

    new_df = df.copy()
    new_df = new_df.rename(columns = new_colnames)        
    new_df['dataset'] = new_auth
    
    new_df.to_csv('dataset_' + old_auth + '.csv', index = False)
    return new_df

## Read Preprocessed Data

In [ ]:
os.chdir(cd)
os.chdir('results')

In [ ]:
weerapana_df = pd.read_csv('weerapana_cravatt_dataset.csv')
backus_df = pd.read_csv('backus_cravatt_dataset.csv')
cao_df = pd.read_csv('cao_backus_dataset.csv')
palafox_df = pd.read_csv('palafox_backus_dataset.csv')
yan_df = pd.read_csv('yan_backus_dataset.csv')
vinogradova_df = pd.read_csv('vinogradova_cravatt_dataset.csv')
kuljanin_df = pd.read_csv('kuljanin_gygi_dataset.csv')
yang_df = pd.read_csv('yang_wang_dataset.csv')
li_df = pd.read_csv('li_yang_dataset.csv')

## Concatonate

In [ ]:
dataset_df = pd.concat([
    weerapana_df, 
    backus_df, 
    cao_df, 
    palafox_df, 
    yan_df, 
    vinogradova_df, 
    kuljanin_df, 
    yang_df, 
    li_df
])

In [ ]:
dataset_df = dataset_df.replace([1.0, 1], 'yes')

In [ ]:
dataset_df = dataset_df.replace([0], None)

In [ ]:
dataset_df = dataset_df.replace(' ', '')

In [ ]:
dataset_df = dataset_df.drop_duplicates()

In [ ]:
dataset_df.to_csv('cysteineomedb_datasetid.csv', index = False)

## Merge by Cysteine ID, Protein ID

In [ ]:
dataset_df = pd.concat([
    weerapana_df, 
    backus_df, 
    cao_df, 
    palafox_df, 
    yan_df, 
    vinogradova_df, 
    kuljanin_df, 
    yang_df, 
    li_df
])

In [ ]:
datasets = [    
    weerapana_df, 
    backus_df, 
    cao_df, 
    palafox_df, 
    yan_df, 
    vinogradova_df, 
    kuljanin_df, 
    yang_df, 
    li_df
]

In [ ]:
dataset_df = dataset_df.replace(' ', '')

In [ ]:
dataset_df = dataset_df.drop_duplicates()

In [ ]:
cys_concat_df = dataset_df[dataset_df['level'] == 'cysteine']
cys_ids_df = cys_concat_df[['cysteineid']]
cys_ids_df = cys_ids_df.drop_duplicates()

In [ ]:
pro_concat_df = dataset_df[dataset_df['level'] == 'protein']
pro_ids_df = pro_concat_df[['proteinid']]
pro_ids_df = pro_ids_df.drop_duplicates()

In [ ]:
cys_ids_df.shape, pro_ids_df.shape

In [ ]:
def get_merged_df (dfs, id_df, cys):
    merged_df = id_df.copy()
    for i in range(len(dfs)):
        current_df = dfs[i]
        if cys == True:
            current_df = current_df[current_df['level'] == 'cysteine']
            current_df = current_df.drop(columns = ['proteinid', 'level', 'dataset'])
            current_df = current_df.rename(columns = {'identified': 'identified_' + str(i), 'identified_datasets': 'identified_datasets_' + str(i),
                                                      'reactive': 'reactive_' + str(i), 'reactive_datasets': 'reactive_datasets_' + str(i),
                                                      'ligandable': 'ligandable_' + str(i), 'ligandable_datasets': 'ligandable_datasets_' + str(i),
                                                      'cellline': 'cellline_' + str(i), 'cellline_datasets': 'cellline_datasets_' + str(i),
                                                      'condition': 'condition_' + str(i), 'condition_datasets': 'condition_datasets_' + str(i),
                                                     })
            merged_df = pd.merge(merged_df, current_df, on = 'cysteineid', how = 'left')
        else:
            current_df = current_df[current_df['level'] == 'protein']
            current_df = current_df.drop(columns = ['cysteineid', 'level', 'dataset'])
            current_df = current_df.rename(columns = {'identified': 'identified_' + str(i), 'identified_datasets': 'identified_datasets_' + str(i),
                                                      'reactive': 'reactive_' + str(i), 'reactive_datasets': 'reactive_datasets_' + str(i),
                                                      'ligandable': 'ligandable_' + str(i), 'ligandable_datasets': 'ligandable_datasets_' + str(i),
                                                      'cellline': 'cellline_' + str(i), 'cellline_datasets': 'cellline_datasets_' + str(i),
                                                      'condition': 'condition_' + str(i), 'condition_datasets': 'condition_datasets_' + str(i),
                                                     })
            merged_df = pd.merge(merged_df, current_df, on = 'proteinid', how = 'left')
        
    return merged_df

In [ ]:
cys_merged_df = get_merged_df(datasets, cys_ids_df, True)

In [ ]:
cys_merged_df

In [ ]:
def get_col_names(df, name, find_in, multi):
    df_cols = df.columns.to_list()
    
    if find_in == True:
        results = [item for item in df_cols if name in item]
    else:
        if multi == True:
            results = [item for item in df_cols if item[0].isupper()]
        else:
            results = [item for item in df_cols if name not in item]
    
    new_results = [ x for x in results if "datasets" not in x ]
    return new_results

In [ ]:
def get_cols(df):
    identified_cols = get_col_names(df, 'identified', True, False)
    reactive_cols = get_col_names(df, 'reactive', True, False)
    ligandable_cols = get_col_names(df, 'ligandable', True, False) 
    cellline_cols = get_col_names(df, 'cellline', True, False)
    condition_cols = get_col_names(df, 'condition', True, False)
    
    return identified_cols, reactive_cols, ligandable_cols, cellline_cols, condition_cols

In [ ]:
identified_cols, reactive_cols, ligandable_cols, cellline_cols, condition_cols = get_cols(cys_merged_df)

In [ ]:
def get_category_datasets(labels):
    category_datasets = []
    for i in range(len(labels)):
            category_dataset = (labels[i].split('_')[0] + '_datasets_' + str(labels[i].split('_')[1]))
            category_datasets.append(category_dataset)
    return category_datasets

In [ ]:
def update_merged_category(df, category, labels):
    datasets = get_category_datasets(labels)
    
    df['category_sum'] = df[labels].sum(axis=1)
    df[category] = np.where(df['category_sum'] >= 1 , 1, 0)
    
    category_datasets = []
    for index, row in df.iterrows():
        current_datasets = []
        for i in range(len(datasets)):
            
            if str(row[datasets[i]]) != 'nan':
                current_datasets.append(row[datasets[i]])
        
        category_datasets.append(list_to_string(list(set(current_datasets)), ';'))
        
            
    df[category + '_datasets'] = category_datasets
    df = df.drop(columns = labels + datasets + ['category_sum'])
    
    return df

In [ ]:
cys_updated_df = update_merged_category(cys_merged_df, 'identified', identified_cols)

In [ ]:
cys_updated_df['proteinid'] = cys_updated_df['cysteineid'].map(lambda x: str(x).split('_')[0]) 

In [ ]:
pro_merged_df = get_merged_df(datasets, pro_ids_df, False)

In [ ]:
pro_updated_df = update_merged_category(pro_merged_df, 'identified', identified_cols)

In [ ]:
cys_updated_df['level'] = 'cysteine'
pro_updated_df['level'] = 'protein'

In [ ]:
updated_df = pd.concat([cys_updated_df, pro_updated_df])

In [ ]:
updated_df = updated_df.replace([1.0, 1], 'yes')
updated_df = updated_df.replace([0], None)

In [ ]:
updated_df.to_csv('cysteineomedb_id.csv', index = False)